<a href="https://colab.research.google.com/github/resh1998/VoterCRM_Backend/blob/main/BackEnd/Data_Collection/Clean_csv_data_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#### Mounting Drive

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir /content/drive/MyDrive/Raw_CSV_files
!mkdir /content/drive/MyDrive/CSV_files

mkdir: cannot create directory ‘/content/drive/MyDrive/Raw_CSV_files’: File exists


#### Pre process all columns (DONT RUN)

In [ ]:
# df = df.str.replace('\n',' ', regex=True)
# df.head()

In [ ]:
# df = df.replace(',' ', regex=True)
# df.head()

#### Function to get data from string (DONT RUN)

In [ ]:
# import re

# def get_data_from_string(text):

#   name = ''
#   relation_name = ''
#   relation_type = ''
#   house_number = ''
#   age = ''
#   gender = ''

#   if 'Husband' in text:

#     match = re.search(r'(?<=Name).*(?=Husband\'s Name)', text)
#     name = match.group()
    
#     match = re.search(r'(?<=Husband\'s Name).*(?=House Number)', text)
#     relation_name = match.group()
    
#     relation_type = 'H'
  
#   elif 'Wife' in text:
    
#     match = re.search(r'(?<=Name).*(?=Wife\'s Name)', text)
#     name = match.group()

#     match = re.search(r'(?<=Wife\'s Name).*(?=House Number)', text)
#     relation_name = match.group()

#     relation_type = 'W'
  
#   elif 'Father' in text:
    
#     match = re.search(r'(?<=Name).*(?=Father\'s Name)', text)
#     name = match.group()

#     match = re.search(r'(?<=Father\'s Name).*(?=House Number)', text)
#     relation_name = match.group()

#     relation_type = 'F'
  
#   match = re.search(r'(?<=House Number).*(?=Age)', text)
#   house_number = match.group()

#   match = re.search(r'(?<=Age).*(?=Gender)', text)
#   age =  match.group()

#   match = re.search(r'(?<=Gender).*(?=)', text)
#   gender = match.group()

#   text_new = name + ',' + relation_name + ',' + relation_type + ',' + house_number + ',' + age + ',' + gender 
  
#   return text_new




#### Text column Split into diff columns

In [5]:
#df['Age'] = df['text'].str.extract(r'((?<=Age)(.|\s)*(?=Gender))')

In [6]:
def process_text_col(df):

  df['text'] = df.text.str.replace('\n', ' ')

  df['text'] = df.text.str.replace('Name ', 'Name')
  df['text'] = df.text.str.replace('Gender ', 'Gender')
  df['text'] = df.text.str.replace('Age ', 'Age')
  df['text'] = df.text.str.replace('House Number ', 'House Number')

  df['Age'] = df['text'].str.extract(r'((?<=Age).*(?=Gender))')
  df['Gender'] = df['text'].str.extract(r'((?<=Gender).*(?=))')   

  def condition(x):
    if 'Husband' in x:
      return 'H'
    elif 'Wife' in x:
      return 'W'
    elif 'Father' in x:
      return 'F'
    elif 'Mother' in x:
      return 'M'
    else:
      return 'G'

  df['Relation_type'] = df['text'].apply(condition)   

  df['House_Number'] = df['text'].str.extract(r'((?<=Number).*(?=Age))')

  df['text'] = df.text.str.replace('Husband', 'Relation')
  df['text'] = df.text.str.replace('Wife', 'Relation')
  df['text'] = df.text.str.replace('Father', 'Relation')
  df['text'] = df.text.str.replace('Mother', 'Relation')
  df['text'] = df.text.str.replace('Other', 'Relation')
  df['text'] = df.text.str.replace('\'s', '')

  df['Name'] = df['text'].str.extract(r'((?<=Name).*(?=Relation))') 

  df['text'] = df['text'].str.split('House').str[0]
  df['text'] = df['text'].str.split('Relation').str[1]

  df['text'] = df.text.str.replace('Name', '')

  df.rename(columns= {'text': 'Relation_Name'}, inplace = True)

  df['Name'] = df['Name'].str.replace('!', 'I')
  df['Name'] = df['Name'].str.replace('\W', ' ', regex = True)

  df['Relation_Name'] = df['Relation_Name'].str.replace('!', 'I')
  df['Relation_Name'] = df['Relation_Name'].str.replace('\W', ' ', regex = True)

  df['House_Number'] = df.House_Number.str.strip()
  df['House_Number'] = df.House_Number.str.replace(r'^\W*', ' ', regex = True)

  df['Age'] = df.Age.str.replace('\W', '', regex = True)
  df['Gender'] = df.Gender.str.replace('\W', '', regex = True)

  return df

#### Voter columns Preprocessing

In [7]:
def process_voter_id_col(df):
  
  df['voter_id'] = df.voter_id.str.replace(' ', '', regex = True)
  
  df['voter_id_no_part'] = df.voter_id.str.findall(r'(\d+)')
  df['voter_id_no_part'] = df.voter_id_no_part.str.join(',').astype(str)

  df['voter_id_no_part'] = df['voter_id_no_part'].str.replace(',', '')

  def add_zeros(x):
    if x is not None and len(x) < 7:
      return x.zfill(7)
    elif x is not None and len(x) > 7:
      return x[-7:]
    else:
      return x

  df['voter_id_no_part'] = df['voter_id_no_part'].apply(add_zeros)

  def new_voter_id_col(num_part, whole_id):
    if num_part is not None and ('G' in whole_id or 'V' in whole_id or 'B' in whole_id):
        return 'GVB' + str(num_part)

    elif num_part is not None:
        return 'TMO' + str(num_part)
    else:
      return whole_id


  df['voter_id_new'] = df.apply(lambda x: new_voter_id_col(x['voter_id_no_part'], x['voter_id']), axis = 1)

  df.drop(['voter_id_no_part', 'voter_id'], axis = 1, inplace = True)

  return df

#### Header Section

In [8]:
def process_header_col(df):

  df['header_section'] = df['header_section'].str.replace('/n', ' ')

  df['Assembly_Constituency_No_and_Name'] = df['header_section'].str.extract(r'((?<=Assembly Constituency No and Name).*(?=Part))')
  df['Assembly_Constituency_No_and_Name'] = df['Assembly_Constituency_No_and_Name'].str.replace(':', '', regex = True)
  df['Assembly_Constituency_No_and_Name'] = df['Assembly_Constituency_No_and_Name'].str.strip()

  df['Section_No_and_Name'] = df['header_section'].str.extract(r'((?<=Section No and Name).*(?=))') 
  df['Section_No_and_Name'] = df['Section_No_and_Name'].str.replace(': ', '')

  df['header_section'] = df['header_section'].str.replace('\n', ' ')

  df['Part_Number'] = df['header_section'].str.extract(r'((?<=Part number).*(?=Section))')

  df['Part_Number'] = df['Part_Number'].str.replace(': ', '').str.strip()

  return df

#### Dataframe Editing

In [11]:
def edit_df(df):

  df = df.drop(['header_section'], axis = 1)
  df = df.rename(columns = {'voter_id_new': 'Voter_id', 'voter_no': 'Voter_no'})

  cols =  ['Voter_no', 'Voter_id', 'Name', 'Relation_Name', 'Relation_type', 'Age', 
         'Gender', 'House_Number', 'Assembly_Constituency_No_and_Name',
          'Section_No_and_Name', 'Part_Number']

  df = df[cols]

  return df

#### Main Function

In [12]:
import pandas as pd
import numpy as np

from pathlib import Path

csv_files = Path("/content/drive/MyDrive/Raw_CSV_files").glob("*.csv")

for csv in csv_files:

  filename = csv.stem
  
  csv_path = '/content/drive/MyDrive/Raw_CSV_files/' + filename + '.csv'

  df = pd.read_csv(csv_path)
  df = process_text_col(df)
  df = process_voter_id_col(df)
  df = process_header_col(df)
  df = edit_df(df)

  csv_path = '/content/drive/MyDrive/CSV_files/' + filename + '.csv'
  df.to_csv(csv_path)
